In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from textblob import TextBlob


In [2]:
# Crear una sesión de Spark
spark = SparkSession.builder.appName("NewsDataInspection").getOrCreate()

# Cargar el archivo CSV
file_path = '../data/raw/news.csv'
df = spark.read.csv(file_path, header=True, inferSchema=True)

24/08/26 23:47:08 WARN Utils: Your hostname, mauro-thinkpad resolves to a loopback address: 127.0.1.1; using 192.168.0.212 instead (on interface wlp0s20f3)
24/08/26 23:47:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/26 23:47:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Contar el número de filas y columnas
num_rows = df.count()
num_cols = len(df.columns)
print(f"\nDimensiones del DataFrame: {num_rows} filas, {num_cols} columnas")


Dimensiones del DataFrame: 402 filas, 3 columnas


In [4]:
# Mostrar las primeras filas del DataFrame
print("Primeras filas del DataFrame:")
df.show(5,truncate=False)

Primeras filas del DataFrame:
+--------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [5]:
# Mostrar el esquema del DataFrame (tipos de datos de cada columna)
print("\nEsquema del DataFrame:")
df.printSchema()


Esquema del DataFrame:
root
 |-- titulo: string (nullable = true)
 |-- link: string (nullable = true)
 |-- cuerpo: string (nullable = true)



In [6]:
# Describir las estadísticas básicas
print("\nDescripción estadística de las columnas numéricas:")
df.describe().show()



Descripción estadística de las columnas numéricas:
+-------+--------------------+--------------------+--------------------+
|summary|              titulo|                link|              cuerpo|
+-------+--------------------+--------------------+--------------------+
|  count|                 401|                 402|                 402|
|   mean|                NULL|                NULL|                NULL|
| stddev|                NULL|                NULL|                NULL|
|    min|21 argentinos com...|https://www.lanac...|"Activision ha an...|
|    max|“Semáforo climber...|https://www.lanac...|“Piensa bien ante...|
+-------+--------------------+--------------------+--------------------+



In [7]:
# Verificar si hay valores nulos
print("\nNúmero de valores nulos por columna:")
df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns]).show()


Número de valores nulos por columna:
+------+----+------+
|titulo|link|cuerpo|
+------+----+------+
|     1|   0|     0|
+------+----+------+



In [8]:
# Filtrar los registros que tienen valores nulos en alguna columna
df_with_nulls = df.filter(F.greatest(*[F.col(c).isNull().cast("int") for c in df.columns]) == 1)

# Mostrar los registros con nulos
print("Registros con valores nulos en alguna columna:")
df_with_nulls.show(truncate=False)

Registros con valores nulos en alguna columna:
+------+----------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|titulo|link                                                                                                                                                            |cuerpo                                                                                                                                                                              

In [9]:
# Eliminar los registros que tienen algún valor nulo
df_cleaned = df.dropna()

In [10]:
# Filtrar los registros que tienen valores nulos en alguna columna
df_with_nulls = df_cleaned.filter(F.greatest(*[F.col(c).isNull().cast("int") for c in df_cleaned.columns]) == 1)

# Mostrar los registros con nulos
print("Registros con valores nulos en alguna columna:")
df_with_nulls.show(truncate=False)

Registros con valores nulos en alguna columna:
+------+----+------+
|titulo|link|cuerpo|
+------+----+------+
+------+----+------+



In [11]:
# Definir una función para calcular el sentimiento usando TextBlob
def get_sentiment(text):
    if text is None:
        return None
    return TextBlob(text).sentiment.polarity

# Registrar la función como una UDF (User Defined Function)
sentiment_udf = F.udf(get_sentiment)

# Crear la columna 'sentiment' basada en el cuerpo de la noticia
df_with_sentiment = df.withColumn("sentiment", sentiment_udf(F.col("cuerpo")))

# Mostrar algunas filas con la nueva columna de sentimiento
print("Primeras filas con puntuación de sentimiento:")
df_with_sentiment.show(3)

Primeras filas con puntuación de sentimiento:


+--------------------+--------------------+--------------------+--------------------+
|              titulo|                link|              cuerpo|           sentiment|
+--------------------+--------------------+--------------------+--------------------+
|Gamescom 2024: vu...|https://www.lanac...|Borderlands 4 el ...|-0.00432900432900...|
|Un juego chino ro...|https://www.lanac...|Cuatro años esper...|-0.14642857142857144|
|En el Mundial de ...|https://www.lanac...|En un evento masi...|0.022222222222222216|
+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows



In [14]:
df_with_sentiment.write.mode("overwrite").csv("../data/processed/news.csv", header=True)

In [15]:
import seaborn as sns

# Asumiendo que df_with_sentiment ya está definido y contiene los datos más recientes

# Paso 1: Monitoreo de Sentimiento por Fecha
df_with_sentiment = df_with_sentiment.withColumn('fecha', F.to_date(F.col('fecha_column_name'), 'yyyy-MM-dd'))
sentiment_by_date = df_with_sentiment.groupBy('fecha').agg(F.avg('sentiment').alias('avg_sentiment')).orderBy('fecha')
sentiment_pd = sentiment_by_date.toPandas()
sentiment_pd = sentiment_pd.sort_values('fecha')

# Visualización
plt.figure(figsize=(14, 7))
sns.lineplot(data=sentiment_pd, x='fecha', y='avg_sentiment', marker='o')
plt.title('Evolución del Sentimiento Promedio de Noticias de Videojuegos')
plt.xlabel('Fecha')
plt.ylabel('Sentimiento Promedio')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('sentiment_trend.png')  # Guarda la gráfica como una imagen
plt.close()

# Paso 10: Sistema de Alerta
threshold = 0.2
sentiment_pd['sentiment_change'] = sentiment_pd['avg_sentiment'].diff()
latest_change = sentiment_pd.iloc[-1]['sentiment_change']
latest_date = sentiment_pd.iloc[-1]['fecha']

if latest_change <= -threshold:
    print(f"ALERTA: El sentimiento promedio cayó {abs(latest_change):.2f} puntos el día {latest_date}.")
    # Aquí podrías añadir código para enviar un correo electrónico o una notificación
else:
    print(f"El sentimiento promedio está estable el día {latest_date}.")


AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `fecha_column_name` cannot be resolved. Did you mean one of the following? [`cuerpo`, `link`, `sentiment`, `titulo`].;
'Project [titulo#17, link#18, cuerpo#19, sentiment#340, to_date('fecha_column_name, Some(yyyy-MM-dd), Some(America/Argentina/Buenos_Aires), false) AS fecha#407]
+- Project [titulo#17, link#18, cuerpo#19, get_sentiment(cuerpo#19)#339 AS sentiment#340]
   +- Relation [titulo#17,link#18,cuerpo#19] csv
